In [2]:
import os
import sys
import pygsheets
import math
import numpy as np
import pandas as pd
import ams_utilities as au
from openpyxl import load_workbook
from itertools import cycle
import scipy
import pyabf
import paq2py
import scipy.stats as ss 
from itertools import product

In [3]:
import pygsheets
from morph_analysis_jake import *
directory = r"\\163.1.248.177\Root\jswann\Morphologies"

In [4]:
#list datapath sheets
datapaths = []
for root, subdirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".xlsx"):
            temppath=os.path.join(root, filename)
            datapaths.append(temppath)
print(len(datapaths))

86


In [5]:
#create dataframe
df_all = {}
for i in datapaths:

    df_single_cell = make_param_df(i)

    if i == datapaths[0]:
        df_all = df_single_cell

    else:

        df_all = pd.concat([df_all, df_single_cell], axis=1, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)

#Sort columns into order and remove zeroes from cell number
# mouseIDs=df_all.columns
# zeroes_removed=[]
# for i in mouseIDs:
#     zeroes_removed.append(i.replace("0", ""))
# df_all.columns=zeroes_removed
df_all=df_all.reindex(sorted(df_all.columns),axis=1)

C:\Users\Packer Lab\Documents\Python Scripts\Code\morph_analysis_jake.py:81: RuntimeWarning: invalid value encountered in double_scalars
  axon_len_SA_ratio = axon_length_total/axon_SA


In [6]:
df_all.head()

,AS016: Cell1,AS016: Cell2,AS029: Cell1,AS030: Cell1,AS035: Cell1,AS036: Cell1,AS038: Cell1,AS039: Cell1,AS039: Cell2,AS040: Cell1,...,AS276: Cell2,AS277: Cell1,AS277: Cell2,AS277: Cell3,AS278: Cell2,AS278: Cell3,AS278: Cell4,AS278: Cell5,AS279: Cell1,AS280: Cell2
Axon_node_total,6.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Axon_length,1620.490000,0.0,28.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Axon_SA,10859.800000,0.0,104.620000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Axon_length_to_SA_ratio,0.149219,NaN,0.271459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Highest_order_axon_segment,4.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:

#Add slice registration data
if 'AP Coords' not in df_all.index:
    df_reg=pd.read_excel('E:\\Jake\\coronal_slice_registration.xlsx', indexCol=0)
    df_reg=df_reg.drop(['Unnamed: 0'],axis=1)
    varNames=['AP Coords', 
             'DV Coords',
             'ML Coords']
    df_reg.index=varNames
    cellID=[]
    for i in df_reg.columns:
        cellID.append(i.replace('Cell0', 'Cell')) 
    df_reg.columns=cellID    
    df_reg.sort_index(axis=1)        
    df_reg=df_reg.drop(['DV Coords', 'ML Coords'])
    df_all=pd.concat([df_all,df_reg], join='outer', sort=True)

df_all

,AS016: Cell1,AS016: Cell2,AS029: Cell1,AS030: Cell1,AS035: Cell1,AS036: Cell1,AS038: Cell1,AS039: Cell1,AS039: Cell2,AS040: Cell1,...,AS276: Cell2,AS277: Cell1,AS277: Cell2,AS277: Cell3,AS278: Cell2,AS278: Cell3,AS278: Cell4,AS278: Cell5,AS279: Cell1,AS280: Cell2
Axon_node_total,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Axon_length,1620.490000,0.000000,28.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Axon_SA,10859.800000,0.000000,104.620000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Axon_length_to_SA_ratio,0.149219,NaN,0.271459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Highest_order_axon_segment,4.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_polar_angle,58.316667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_polar_angle_std,38.830256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_local_angle,80.433333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_local_angle_std,40.927216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_spline_angle,68.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Add polar analysis data
if 'Dendrite_polar_length_mean' not in df_all.index:
    df_polar=pd.read_excel('E:\\Jake\\Morphologies_PolarMetrics.xlsx', indexCol=0)
    df_polar=df_polar.drop(['Unnamed: 0'],axis=1)
    variableNames=['Dendrite_polar_length_mean', 
             'Dendrite_polar_length_std',
             'Dendrite_polar_length_variance',
             'Dendrite_polar_longest_bin']
    df_polar.index=variableNames
    df_all=pd.concat([df_all,df_polar], join='inner') #This will drop any cells with no dendritic data
df_polar.T

,Dendrite_polar_length_mean,Dendrite_polar_length_std,Dendrite_polar_length_variance,Dendrite_polar_longest_bin
AS016: Cell1,0.398822,0.211449,0.044711,260.0
AS016: Cell2,0.430521,0.252960,0.063989,140.0
AS029: Cell1,0.374605,0.300087,0.090052,340.0
AS030: Cell1,0.327464,0.236692,0.056023,100.0
AS035: Cell1,0.321695,0.279975,0.078386,340.0
AS036: Cell1,0.373783,0.257125,0.066113,140.0
AS038: Cell1,0.459234,0.181563,0.032965,100.0
AS039: Cell1,0.379131,0.255947,0.065509,340.0
AS039: Cell2,0.638801,0.186113,0.034638,300.0
AS040: Cell1,0.345150,0.238962,0.057103,180.0


In [9]:
df_all.head()

,AS016: Cell1,AS016: Cell2,AS029: Cell1,AS030: Cell1,AS035: Cell1,AS036: Cell1,AS038: Cell1,AS039: Cell1,AS039: Cell2,AS040: Cell1,...,AS276: Cell2,AS277: Cell1,AS277: Cell2,AS277: Cell3,AS278: Cell2,AS278: Cell3,AS278: Cell4,AS278: Cell5,AS279: Cell1,AS280: Cell2
Axon_node_total,6.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Axon_length,1620.490000,0.0,28.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Axon_SA,10859.800000,0.0,104.620000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Axon_length_to_SA_ratio,0.149219,NaN,0.271459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Highest_order_axon_segment,4.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#add slice orientations
if 'Slice_Orientation' not in df_all.index:
    text_file=open('E:\\Jake\\Orientations.txt')
    orientations=list(text_file.read())
    orientations=pd.Series(orientations).to_frame().T
    orientations.index=['Slice_Orientation']
    orientations.columns=df_all.columns
    df_all=pd.concat([orientations,df_all], join='inner')

In [11]:
#export to google sheet
creds = r"C:\Users\Packer Lab\Documents\Python Scripts\Code\creds_jake.json"
gc = pygsheets.authorize(service_file=creds)

df = pd.DataFrame()

params = list(df_all.index.values)
df['Parameters'] = params
df.columns = ['Mouse#: Cell']

sh = gc.open('Morph_analysis')

wks = sh[0]
wks.set_dataframe(df, (1,1))
wks.set_dataframe(df_all, (1,2))

In [12]:
df_all

,AS016: Cell1,AS016: Cell2,AS029: Cell1,AS030: Cell1,AS035: Cell1,AS036: Cell1,AS038: Cell1,AS039: Cell1,AS039: Cell2,AS040: Cell1,...,AS276: Cell2,AS277: Cell1,AS277: Cell2,AS277: Cell3,AS278: Cell2,AS278: Cell3,AS278: Cell4,AS278: Cell5,AS279: Cell1,AS280: Cell2
Slice_Orientation,H,H,H,H,H,C,H,H,H,H,...,C,C,C,C,C,C,C,C,C,C
Axon_node_total,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Axon_length,1620.49,0,28.4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Axon_SA,10859.8,0,104.62,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Axon_length_to_SA_ratio,0.149219,NaN,0.271459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Highest_order_axon_segment,4,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_polar_angle,58.3167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_polar_angle_std,38.8303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_local_angle,80.4333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Axon_local_angle_std,40.9272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
len(df_reg.columns)

69

In [14]:
for i in df_all.columns:
    if i not in df_reg.columns:
        print(i)

AS016: Cell1
AS016: Cell2
AS029: Cell1
AS030: Cell1
AS055: Cell1
AS062: Cell3
AS087: Cell1
AS091: Cell2
AS091: Cell4
AS108: Cell1
AS109: Cell1
AS113: Cell2
AS116: Cell1
AS116: Cell2
AS132: Cell3
AS143: Cell3
AS152: Cell1
